In [ ]:
import numpy as np
import re
import math

In [ ]:
with open('./words.txt') as f:
    words = f.read().splitlines()
    cleaned_words = [word.lower() for word in words if ',' not in word and '.' not in word]

In [ ]:
def generate_palindrome(word_len, corpus):
    # init the palindrome
    potential_words = [word for word in corpus if len(word) == word_len]
    print(f"{len(potential_words)} {word_len} letter words available")
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    possible_solutions = [list(alphabet) for i in range(word_len)]

    while True:
        # get char to collapse
        condition = np.array([len(char_options) for char_options in possible_solutions]) != 1
        collapsable_indices = np.where(condition)[0]
        index_to_collapse = np.random.choice(collapsable_indices)

        # collapse the character and the corresponding char for the pallindrome
        character_choice = np.random.choice(possible_solutions[index_to_collapse])
        possible_solutions[index_to_collapse] = [character_choice]
        possible_solutions[word_len - 1 - index_to_collapse] = [character_choice]

        # get remaining valid corpus
        template_word = ['.'] * word_len
        for i in range(word_len):
            if len(possible_solutions[i]) == 1:
                template_word[i] = possible_solutions[i][0]
        pattern = ''.join(template_word)
        regex = re.compile(pattern)

        potential_words = [word for word in potential_words if regex.match(word)]
        print(f"{len(potential_words)} words available matching {pattern}")

        # update character options
        for i in range(math.ceil(word_len/2)):
            if len(possible_solutions[i]) != 1:
                potential_chars = set([word[i] for word in potential_words])
                compliment_potential_chars = set([word[word_len - 1 - i] for word in potential_words])
                new_character_choices = potential_chars & compliment_potential_chars
                if len(new_character_choices) == 0:
                    raise RuntimeError("Generation Failed")
                possible_solutions[i] = list(new_character_choices)
                possible_solutions[word_len - 1 - i] = list(new_character_choices)

        # if we are done => break
        if np.all([len(char_options) == 1 for char_options in possible_solutions]):
            break

    final_word = [arr[0] for arr in possible_solutions]
    return ''.join(final_word)

In [ ]:
final_word = ''
while final_word == '':
    try:
        final_word = generate_palindrome(7, cleaned_words)
    except:
        print("Failed To Generate Restarting...")
        continue
print(f"Final Word {final_word}")